# Example of Collecting Data from multiple files

In [ ]:
!pip install nltk

In [ ]:
!pip install wordcloud

In [ ]:
!pip install textblob

In [ ]:
!pip install flask

In [ ]:
import nltk
nltk.download("popular")
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

In [ ]:
import os
import pandas as pd
import nltk
path="data/reuters_data/"
files=os.listdir(path)
files

In [ ]:
f=open(os.path.join(path, files[1]),'r',encoding='latin-1')

In [ ]:
text=""
for line in f:
    if line.strip()=="":continue #if line is blank skip it
    else:
        text+=' '+line.strip() #else append all lines
print(text)
f.close()

In [ ]:
target=[]
article_text=[]
for file in files:
    if '.txt' not in file:continue
    f=open(os.path.join(path,file),
           encoding='latin-1')
    article_text.append(" ".join([line.strip() for line in
                                       f if line.strip()!=""]))
    if "crude" in file:
        target.append("crude")
    else:
        target.append("money")
    f.close()

In [ ]:
mydata=pd.DataFrame({'target':target,'article_text':article_text})
mydata.shape

In [ ]:
mydata.sample(5)

# Text Data Exploration with word cloud

Text data visualisation. Words with high frequency appear larger and words with low frequency appear smaller. 

In [ ]:
from wordcloud import WordCloud

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
all_articles=' '.join(mydata['article_text']) #all articles combined into one single string

In [ ]:
crude_articles=" ".join(mydata.loc[mydata['target']=='crude',
                                   'article_text']) #all crude articles combined into one single string

In [ ]:
money_articles=" ".join(mydata.loc[mydata['target']=='money',
                                   'article_text']) #all money articles combined into one single string

In [ ]:
wordcloud = WordCloud().generate(all_articles)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

In [ ]:
wordcloud = WordCloud().generate(crude_articles)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

In [ ]:
wordcloud = WordCloud().generate(money_articles)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# Text Data Exploration with nltk

In [ ]:
from nltk import word_tokenize

In [ ]:
tokens=word_tokenize(money_articles) #Breaking down the text into individual words
tokens

In [ ]:
money_articles_Text=nltk.Text(tokens) #nltk based data structure
money_articles_Text, type(money_articles_Text)

In [ ]:
money_articles_Text.concordance('dollar') #for understanding the context of given words, as in where they are used in to article
#money_articles_Text.concordance('dlr')
#money_articles_Text.concordance('bank')

In [ ]:
money_articles_Text.similar('bank') #this is derived based on common contexts. Not as efficient as human reasoning.

In [ ]:
money_articles_Text.common_contexts(['dollar','dlr'])

# Text Data features with tfidf 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize

lemma = WordNetLemmatizer() #for combining similar words together

my_stop=set(stopwords.words('english')+list(punctuation))

In [ ]:
lemma.lemmatize("peoples")

In [ ]:
my_stop #these words will not be used for analysis

In [ ]:
def split_into_lemmas(message):
    message=message.lower()
    words = word_tokenize(message)
    words_sans_stop=[]
    for word in words :
        if word in my_stop:continue
        words_sans_stop.append(word)
    return [lemma.lemmatize(word) for word in words_sans_stop]

In [ ]:
tfidf= TfidfVectorizer(analyzer=split_into_lemmas,
                       min_df=20,
                       max_df=500,
                       stop_words=my_stop)
#min no of occurence for a word to be considered for analysis is min_df = 50
#max_df is said to ignore the very common words that are redundant for analysis

In [ ]:
tfidf.fit(mydata['article_text'])

In [ ]:
tfidf_data=tfidf.transform(mydata['article_text'])

In [ ]:
tfidf_data.shape

In [ ]:
tfidf_data

# Sentiment Analysis with nltk

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer=SentimentIntensityAnalyzer()

In [ ]:
sentences = ["VADER is smart, handsome, and funny.",      # positive sentence example
            "VADER is not smart, handsome, nor funny.",   # negation sentence example
            "VADER is smart, handsome, and funny!",       # punctuation emphasis handled correctly (sentiment intensity adjusted)
            "VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
            "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
            "VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
            "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!",# booster words & punctuation make this close to ceiling for score
            "The book was good.",                                     # positive sentence
            "The book was kind of good.",                 # qualified positive sentence is handled correctly (intensity adjusted)
        '''The plot was good, but the characters are uncompelling and 
             the dialog is not great.''', # mixed negation sentence
            "At least it isn't a horrible book.",         # negated negative sentence with contraction
            "Make sure you :) or :D today!",              # emoticons handled
            "Today SUX!",                                 # negative slang with capitalization emphasis
            "Today only kinda sux! But I'll get by, lol"  # mixed sentiment example with slang and constrastive conjunction "but"
             ]

In [ ]:
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print(sentence, str(vs))

# Language Correction, Detection and Translation with TextBlob

In [ ]:
from textblob import TextBlob 

In [ ]:
b = TextBlob("I havv goood color !")
b.correct(), b.detect_language()

In [ ]:
chinese_blob = TextBlob(u"美丽优于丑陋")

In [ ]:
chinese_blob.translate(from_lang="zh-CN", to='en')

In [ ]:
b = TextBlob(u"بسيط هو أفضل من مجمع")

In [ ]:
b.detect_language()

# Part Of Speech (POS) Tagging

source : https://stackoverflow.com/questions/1833252/java-stanford-nlp-part-of-speech-labels

* CC: conjunction, coordinating

    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
    
* CD: numeral, cardinal

    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
    
* DT: determiner

    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
    
* EX: existential there
    there
    
* FW: foreign word

    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
    
* IN: preposition or conjunction, subordinating

    astride among uppon whether out inside pro despite on by throughout
    below within for towards near behind atop around if like until below
    next into if beside ...
    
* JJ: adjective or numeral, ordinal

    third ill-mannered pre-war regrettable oiled calamitous first separable
    ectoplasmic battery-powered participatory fourth still-to-be-named
    multilingual multi-disciplinary ...
    
* JJR: adjective, comparative

    bleaker braver breezier briefer brighter brisker broader bumper busier
    calmer cheaper choosier cleaner clearer closer colder commoner costlier
    cozier creamier crunchier cuter ...
    
* JJS: adjective, superlative

    calmest cheapest choicest classiest cleanest clearest closest commonest
    corniest costliest crassest creepiest crudest cutest darkest deadliest
    dearest deepest densest dinkiest ...
    
* LS: list item marker

    A A. B B. C C. D E F First G H I J K One SP-44001 SP-44002 SP-44005
    SP-44007 Second Third Three Two * a b c d first five four one six three
    two
    
* MD: modal auxiliary

    can cannot could couldn't dare may might must need ought shall should
    shouldn't will would
    
* NN: noun, common, singular or mass

    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
    
* NNS: noun, common, plural

    undergraduates scotches bric-a-brac products bodyguards facets coasts
    divestitures storehouses designs clubs fragrances averages
    subjectivists apprehensions muses factory-jobs ...
    
* NNP: noun, proper, singular

    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
    
* NNPS: noun, proper, plural

    Americans Americas Amharas Amityvilles Amusements Anarcho-Syndicalists
    Andalusians Andes Andruses Angels Animals Anthony Antilles Antiques
    Apache Apaches Apocrypha ...
    
* PDT: pre-determiner

    all both half many quite such sure this
    
* POS: genitive marker
    ' 's

* PRP: pronoun, personal

    hers herself him himself hisself it itself me myself one oneself ours
    ourselves ownself self she thee theirs them themselves they thou thy us
    
* PRP$': pronoun, possessive

    her his mine my our ours their thy your
    
* RB: adverb

    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
    
* RBR: adverb, comparative

    further gloomier grander graver greater grimmer harder harsher
    healthier heavier higher however larger later leaner lengthier less-
    perfectly lesser lonelier longer louder lower more ...
    
* RBS: adverb, superlative

    best biggest bluntest earliest farthest first furthest hardest
    heartiest highest largest least less most nearest second tightest worst
    
* RP: particle

    aboard about across along apart around aside at away back before behind
    by crop down ever fast for forth from go high i.e. in into just later
    low more off on open out over per pie raising start teeth that through
    under unto up up-pp upon whole with you
    
* SYM: symbol

    % & ' '' ''. ) ). * + ,. < = > @ A[fj] U.S U.S.S.R * ** ***
    
* TO: "to" as preposition or infinitive marker

    to
    
* UH: interjection

    Goodbye Goody Gosh Wow Jeepers Jee-sus Hubba Hey Kee-reist Oops amen
    huh howdy uh dammit whammo shucks heck anyways whodunnit honey golly
    man baby diddle hush sonuvabitch ...
    
* VB: verb, base form

    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...
    
* VBD: verb, past tense

    dipped pleaded swiped regummed soaked tidied convened halted registered
    cushioned exacted snubbed strode aimed adopted belied figgered
    speculated wore appreciated contemplated ...
    
* VBG: verb, present participle or gerund

    telegraphing stirring focusing angering judging stalling lactating
    hankerin' alleging veering capping approaching traveling besieging
    encrypting interrupting erasing wincing ...
    
* VBN: verb, past participle

    multihulled dilapidated aerosolized chaired languished panelized used
    experimented flourished imitated reunifed factored condensed sheared
    unsettled primed dubbed desired ...
    
* VBP: verb, present tense, not 3rd person singular

    predominate wrap resort sue twist spill cure lengthen brush terminate
    appear tend stray glisten obtain comprise detest tease attract
    emphasize mold postpone sever return wag ...
    
* VBZ: verb, present tense, 3rd person singular

    bases reconstructs marks mixes displeases seals carps weaves snatches
    slumps stretches authorizes smolders pictures emerges stockpiles
    seduces fizzes uses bolsters slaps speaks pleads ...
    
* WDT: WH-determiner
    that what whatever which whichever
    
* WP: WH-pronoun

    that what whatever whatsoever which who whom whosoever
    
* WP$: WH-pronoun, possessive

    whose
    
* WRB: Wh-adverb

    how however whence whenever where whereby whereever wherein whereof why

In [ ]:
sentence = "We hope you love your sleepycat mattress. Welcom to the sleepycat community."

In [ ]:
#nltk.download('averaged_perceptron_tagger')

In [ ]:
nltk.pos_tag(nltk.word_tokenize(sentence))